In [3]:
import numpy as np
from collections import defaultdict


def parse_metrics_line(line):
    metrics = line.strip().split(',')
    return {metric.split(':')[0].strip(): float(metric.split(':')[1].strip()) for metric in metrics}

def process_results_file(file_path):
    results = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

    with open(file_path, 'r') as f:
        lines = f.readlines()

    
    for i in range(0, len(lines), 3):
        if i+1 >= len(lines):  
            break
        if lines[i].strip() == '':  
            continue

        try:
            model_info = lines[i].strip().split()
            metrics_info = lines[i+1].strip()
            # print(model_info )
            # print(metrics_info)
        except ValueError as e:
            print(f"ValueError at line {i}: {e}")
            continue


        if len(model_info) >= 3:
            _, model_name, input_length = model_info[0], model_info[1], model_info[2]
            input_length = int(input_length)
            metrics = parse_metrics_line(metrics_info)

            for metric_name, metric_value in metrics.items():
                results[model_name][input_length][metric_name].append(metric_value)
        else:
            print(f"Skipped line {i} due to incorrect format.")


    average_metrics = {model: {} for model in results}
    for model in results:
        for length in results[model]:
            average_metrics[model][length] = {
                metric_name: np.mean(values)
                for metric_name, values in results[model][length].items()
            }

    return average_metrics

file_path = 'result_Fisher.txt' 
average_metrics = process_results_file(file_path)

for model, lengths_data in average_metrics.items():
    print(f"Model: {model}")
    for length, metrics in lengths_data.items():
        print(f"  Input Length: {length}, Average Metrics: {metrics}")

Model: harmony
  Input Length: 72, Average Metrics: {'mse': 0.045293026603758335, 'crps': 0.09952614717185497}
  Input Length: 144, Average Metrics: {'mse': 0.04575974158942699, 'crps': 0.09399256221950054}
  Input Length: 288, Average Metrics: {'mse': 0.047408952936530115, 'crps': 0.10965233370661735}
